In [67]:
import pandas as pd
import numpy as np
import ast


In [43]:
df = pd.read_excel('BA_Stock news with sentiment.xlsx')

### Уже с распознанными сущностями

In [120]:
df = pd.read_csv('BA_with_entities.csv')

In [121]:
df

,date,content,negative,neutral,positive,entities
0,2024-04-24,Boeing has been mired in a safety crisis since...,0.981807,0.017569,0.000624,['boeing']
1,2024-04-24,Families of the victims of two fatal Boeing 73...,0.653259,0.345061,0.001681,"['u', 'u', '.', '.', 's', 's', '. justice depa..."
2,2024-04-24,"Merle Meyers, who left Boeing last year after ...",0.000208,0.999556,0.000236,['boeing']
3,2024-04-24,Japan Airlines will continue using both Airbus...,0.000074,0.999808,0.000118,"['japan airlines', 'airbus', 'airbus', 'boeing']"
4,2024-04-24,"Bert Subin, Aerospace Analyst at Stifel, gives...",0.000277,0.000226,0.999497,"['st', 'st', '##ife', '##ife', '##l', '##l', '..."
...,...,...,...,...,...,...
9018,2016-02-26,"NEW YORK, NY--(Marketwired - February 26, 2016...",0.000131,0.999807,0.000062,"['& ##insky', 'usd', 'usd', '##c', '##c', 'boe..."
9019,2016-02-24,"NEW YORK, Feb. 24, 2016 (GLOBE NEWSWIRE) -- ...",0.000141,0.999800,0.000060,"['globe news ##wire', 'levi & ko ##rs ##insky'..."
9020,2016-02-24,"SAN FRANCISCO, Feb. 24, 2016 (GLOBE NEWSWIRE...",0.857394,0.141325,0.001281,"['globe news', 'hagen', 'hagen', '##s berman s..."
9021,2016-02-22,"NEW YORK, Feb. 22, 2016 (GLOBE NEWSWIRE) -- ...",0.000274,0.999333,0.000393,"['globe news ##wire', 'po', 'po', '##mer', '##..."


In [129]:
def reconstruct_entities(ner_results):
    reconstructed_entities = []
    current_word = ""
    current_entity = None

    for result in ner_results:
        token = result['word'].replace("##", "")

        if result['entity'].startswith("B-"):
            if current_word:  # Заканчиваем предыдущее слово
                reconstructed_entities.append(
                    {"word": current_word, "entity": current_entity}
                )
            current_word = token  # Начинаем новое слово
            current_entity = result['entity']
        elif result['entity'].startswith("I-") and current_word:
            current_word += token  # Присоединяем к текущему слову

    # Добавляем последнее слово, если оно есть
    if current_word:
        reconstructed_entities.append({"word": current_word, "entity": current_entity})

    return reconstructed_entities


print("Объединенные результаты:")
for org in reconstructed_orgs:
    print(org)


Объединенные результаты:
{'word': 'st', 'entity': 'B-ORG'}
{'word': 'ife', 'entity': 'B-ORG'}
{'word': 'l', 'entity': 'B-ORG'}
{'word': 'boeing', 'entity': 'B-ORG'}


In [44]:
df = df.dropna(subset=['content'])

In [45]:
df = df.drop(columns=['publisher', 'url', 'title'])
df

,date,content,negative,neutral,positive
0,2024-04-24,Boeing has been mired in a safety crisis since...,0.981807,0.017569,0.000624
1,2024-04-24,Families of the victims of two fatal Boeing 73...,0.653259,0.345061,0.001681
2,2024-04-24,"Merle Meyers, who left Boeing last year after ...",0.000208,0.999556,0.000236
3,2024-04-24,Japan Airlines will continue using both Airbus...,0.000074,0.999808,0.000118
4,2024-04-24,"Bert Subin, Aerospace Analyst at Stifel, gives...",0.000277,0.000226,0.999497
...,...,...,...,...,...
9038,2016-02-26,"NEW YORK, NY--(Marketwired - February 26, 2016...",0.000131,0.999807,0.000062
9039,2016-02-24,"NEW YORK, Feb. 24, 2016 (GLOBE NEWSWIRE) -- ...",0.000141,0.999800,0.000060
9040,2016-02-24,"SAN FRANCISCO, Feb. 24, 2016 (GLOBE NEWSWIRE...",0.857394,0.141325,0.001281
9041,2016-02-22,"NEW YORK, Feb. 22, 2016 (GLOBE NEWSWIRE) -- ...",0.000274,0.999333,0.000393


## Загрузка Fine_Tune модели для NER

In [122]:
from transformers import BertTokenizer, BertForTokenClassification, AutoConfig, pipeline
from safetensors.torch import load_file
import os


In [123]:
from transformers import pipeline, BertTokenizer, BertForTokenClassification, AutoConfig
from safetensors.torch import load_file

# Загрузка конфигурации и модели
config_path = "config.json"
config = AutoConfig.from_pretrained(config_path)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Загрузка модели с конфигурацией
model = BertForTokenClassification.from_pretrained(
    None,
    state_dict=load_file("model.safetensors"),
    config=config
)

# Создание pipeline для NER
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

# Тестовое предложение
example = "North Airlines, Google and Microsoft are leading tech companies. IBM is also a major player."

# Предсказание NER
ner_results = nlp(example)

# Фильтрация результатов, оставляя только B-ORG и I-ORG
org_results = [result for result in ner_results if result['entity'] in ['B-ORG', 'I-ORG']]

# Вывод отфильтрованных результатов
print("Результаты NER (только B-ORG и I-ORG):")
for result in org_results:
    print(result)


Результаты NER (только B-ORG и I-ORG):
{'entity': 'B-ORG', 'score': 0.9873624, 'index': 1, 'word': 'north', 'start': None, 'end': None}
{'entity': 'I-ORG', 'score': 0.97545075, 'index': 2, 'word': 'airlines', 'start': None, 'end': None}
{'entity': 'B-ORG', 'score': 0.9943715, 'index': 4, 'word': 'google', 'start': None, 'end': None}
{'entity': 'B-ORG', 'score': 0.99384284, 'index': 6, 'word': 'microsoft', 'start': None, 'end': None}
{'entity': 'B-ORG', 'score': 0.99315506, 'index': 12, 'word': 'ibm', 'start': None, 'end': None}


In [130]:
df['content'].iloc[4]

"Bert Subin, Aerospace Analyst at Stifel, gives his expectations for Boeing's earnings report."

In [142]:
ner_results = nlp(df['content'].iloc[4])
ner_results

[{'entity': 'B-PER',
  'score': 0.9978955,
  'index': 1,
  'word': 'bert',
  'start': None,
  'end': None},
 {'entity': 'I-PER',
  'score': 0.9980567,
  'index': 2,
  'word': 'sub',
  'start': None,
  'end': None},
 {'entity': 'I-PER',
  'score': 0.99783415,
  'index': 3,
  'word': '##in',
  'start': None,
  'end': None},
 {'entity': 'B-ORG',
  'score': 0.9961008,
  'index': 8,
  'word': 'st',
  'start': None,
  'end': None},
 {'entity': 'B-ORG',
  'score': 0.9957217,
  'index': 9,
  'word': '##ife',
  'start': None,
  'end': None},
 {'entity': 'B-ORG',
  'score': 0.99582916,
  'index': 10,
  'word': '##l',
  'start': None,
  'end': None},
 {'entity': 'B-ORG',
  'score': 0.9918213,
  'index': 16,
  'word': 'boeing',
  'start': None,
  'end': None}]

In [141]:
df['entities'].iloc[4]

"['st', 'st', '##ife', '##ife', '##l', '##l', 'boeing']"

In [131]:
# Тестовое предложение
example = df['content'].iloc[4]

# Предсказание NER
ner_results = nlp(example)

# Фильтрация результатов, оставляя только B-ORG и I-ORG
org_results = [result for result in ner_results if result['entity'] in ['B-ORG', 'I-ORG']]

# Вывод отфильтрованных результатов
print("Результаты NER (только B-ORG и I-ORG):")
for result in org_results:
    print(result)

Результаты NER (только B-ORG и I-ORG):
{'entity': 'B-ORG', 'score': 0.9961008, 'index': 8, 'word': 'st', 'start': None, 'end': None}
{'entity': 'B-ORG', 'score': 0.9957217, 'index': 9, 'word': '##ife', 'start': None, 'end': None}
{'entity': 'B-ORG', 'score': 0.99582916, 'index': 10, 'word': '##l', 'start': None, 'end': None}
{'entity': 'B-ORG', 'score': 0.9918213, 'index': 16, 'word': 'boeing', 'start': None, 'end': None}


In [135]:
org_results = [result for result in ner_results if result['entity'] in ['B-ORG', 'I-ORG']]
org_results

[{'entity': 'B-ORG',
  'score': 0.9961008,
  'index': 8,
  'word': 'st',
  'start': None,
  'end': None},
 {'entity': 'B-ORG',
  'score': 0.9957217,
  'index': 9,
  'word': '##ife',
  'start': None,
  'end': None},
 {'entity': 'B-ORG',
  'score': 0.99582916,
  'index': 10,
  'word': '##l',
  'start': None,
  'end': None},
 {'entity': 'B-ORG',
  'score': 0.9918213,
  'index': 16,
  'word': 'boeing',
  'start': None,
  'end': None}]

In [138]:
org_results[0]

{'entity': 'B-ORG',
 'score': 0.9961008,
 'index': 8,
 'word': 'st',
 'start': None,
 'end': None}

In [139]:
org_results[1]

{'entity': 'B-ORG',
 'score': 0.9957217,
 'index': 9,
 'word': '##ife',
 'start': None,
 'end': None}

In [ ]:
stack = []
for i in range(len(org_results)):
    if stack == [] and not org_results[i]['word'].startswith('##'):
        if org_results[i]['entity'] == 'B-ORG' and not org_results[i]['word'].startswith('##'):
            stack.append(org_results[i]['word'])
    elif org_results[i]['entity'] == 'B-ORG' and org_results[i]['word'].startswith('##'):
        stack.append(org_results[i]['word'])




In [48]:
# def find_entity(example):
#     ner_results = nlp(example)
#     org_results = [result for result in ner_results if result['entity'] in ['B-ORG', 'I-ORG']]
    
#     entities = []
#     stack = []
#     # Пройдемся по списку, избегая выхода за пределы индекса
#     for i in range(len(org_results)):
#         entity = org_results[i]['entity']
#         word = org_results[i]['word']
        
#         if entity == 'B-ORG':
#             # Если начало новой организации, очистим стек
#             stack = [word]
#             # Если следующий элемент - тоже 'B-ORG', добавим текущий стек в 'entities'
#             if i + 1 < len(org_results) and org_results[i + 1]['entity'] == 'B-ORG':
#                 entities.append(' '.join(stack))
#         elif entity == 'I-ORG':
#             # Если продолжение организации, добавим в стек
#             stack.append(word)

#         # Если конец списка или следующий элемент - новое начало, добавляем текущий стек в 'entities'
#         if i + 1 >= len(org_results) or org_results[i + 1]['entity'] == 'B-ORG':
#             entities.append(' '.join(stack))
#     return entities

In [155]:
def find_entity(example):
    ner_results = nlp(example)
    org_results = [result for result in ner_results if result['entity'].startswith('ORG')]
    
    entities = []
    current_word = ""
    current_entity = None
    current_start = None
    
    # Проходим по списку результатов NER
    for i, result in enumerate(org_results):
        word = result['word'].replace("##", "")  # Удаляем префикс '##'
        entity = result['entity']
        
        if entity.startswith("B-"):
            # Если это новая сущность, добавляем предыдущую, если она есть
            if current_word:
                entities.append(current_word)
            # Начинаем новую сущность
            current_word = word
        elif entity.startswith("I-"):
            # Если это продолжение той же сущности, добавляем пробел и слово
            current_word += word
    
    # Добавляем последнее слово, если оно есть
    if current_word:
        entities.append(current_word)

    return entities


In [145]:
df = df[0:500]

In [156]:
df['entities_2'] = np.nan
df['entities_2'] = df['content'].apply(find_entity)

/var/folders/yj/nhkf3nv505d5kh0dsk42f3ym0000gn/T/ipykernel_19637/2580237980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['entities_2'] = np.nan
/var/folders/yj/nhkf3nv505d5kh0dsk42f3ym0000gn/T/ipykernel_19637/2580237980.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['entities_2'] = df['content'].apply(find_entity)


In [157]:
df

,date,content,negative,neutral,positive,entities,entities_2
0,2024-04-24,Boeing has been mired in a safety crisis since...,0.981807,0.017569,0.000624,['boeing'],[]
1,2024-04-24,Families of the victims of two fatal Boeing 73...,0.653259,0.345061,0.001681,"['u', 'u', '.', '.', 's', 's', '. justice depa...",[]
2,2024-04-24,"Merle Meyers, who left Boeing last year after ...",0.000208,0.999556,0.000236,['boeing'],[]
3,2024-04-24,Japan Airlines will continue using both Airbus...,0.000074,0.999808,0.000118,"['japan airlines', 'airbus', 'airbus', 'boeing']",[]
4,2024-04-24,"Bert Subin, Aerospace Analyst at Stifel, gives...",0.000277,0.000226,0.999497,"['st', 'st', '##ife', '##ife', '##l', '##l', '...",[]
...,...,...,...,...,...,...,...
495,2024-02-15,"Boeing's margins are expanding, and cash flow ...",0.000205,0.000099,0.999696,['boeing'],[]
496,2024-02-15,Airbus has announced a special dividend follow...,0.000257,0.000280,0.999463,['airbus'],[]
497,2024-02-15,Delta CEO Ed Bastian told FOX Business that Bo...,0.000171,0.999746,0.000084,"['delta', 'delta', 'fox business', 'boeing', '...",[]
498,2024-02-14,Thai Airways International said on Wednesday i...,0.000269,0.000191,0.999540,"['thai airways international', 'boeing', 'boei...",[]


In [50]:
df['content'][4]

"Bert Subin, Aerospace Analyst at Stifel, gives his expectations for Boeing's earnings report."

In [51]:
df['entities'][4]

"['st', 'st', '##ife', '##ife', '##l', '##l', 'boeing']"

In [71]:
# df.to_csv('BA_with_entities.csv', index=False) 

In [119]:
df = pd.read_csv('BA_with_entities.csv')

In [70]:
df

,date,content,negative,neutral,positive,entities
0,2024-04-24,Boeing has been mired in a safety crisis since...,0.981807,0.017569,0.000624,['boeing']
1,2024-04-24,Families of the victims of two fatal Boeing 73...,0.653259,0.345061,0.001681,"['u', 'u', '.', '.', 's', 's', '. justice depa..."
2,2024-04-24,"Merle Meyers, who left Boeing last year after ...",0.000208,0.999556,0.000236,['boeing']
3,2024-04-24,Japan Airlines will continue using both Airbus...,0.000074,0.999808,0.000118,"['japan airlines', 'airbus', 'airbus', 'boeing']"
4,2024-04-24,"Bert Subin, Aerospace Analyst at Stifel, gives...",0.000277,0.000226,0.999497,"['st', 'st', '##ife', '##ife', '##l', '##l', '..."
...,...,...,...,...,...,...
9018,2016-02-26,"NEW YORK, NY--(Marketwired - February 26, 2016...",0.000131,0.999807,0.000062,"['& ##insky', 'usd', 'usd', '##c', '##c', 'boe..."
9019,2016-02-24,"NEW YORK, Feb. 24, 2016 (GLOBE NEWSWIRE) -- ...",0.000141,0.999800,0.000060,"['globe news ##wire', 'levi & ko ##rs ##insky'..."
9020,2016-02-24,"SAN FRANCISCO, Feb. 24, 2016 (GLOBE NEWSWIRE...",0.857394,0.141325,0.001281,"['globe news', 'hagen', 'hagen', '##s berman s..."
9021,2016-02-22,"NEW YORK, Feb. 22, 2016 (GLOBE NEWSWIRE) -- ...",0.000274,0.999333,0.000393,"['globe news ##wire', 'po', 'po', '##mer', '##..."


In [71]:
def str_tolist(text):
    actual_list = ast.literal_eval(text)
    return actual_list

In [72]:
df['entities'] = df['entities'].apply(str_tolist)
df['entities'][3]

['japan airlines', 'airbus', 'airbus', 'boeing']

In [73]:
df_stox = pd.read_csv('STOXX600.csv', sep=';')
df_stox = df_stox[['code', 'name']]
df_stox = df_stox.reset_index(drop=True)
df_stox.head(5)

,code,name
0,1COV.DE,COVESTRO AG
1,A2A.MI,A2A
2,AAK.ST,AAK
3,AAL.L,ANGLO AMERICAN PLC
4,AALB.AS,AALBERTS NV


In [74]:
df_sp500 = pd.read_csv('SP500.csv', sep=';')
df_sp500 = df_sp500[['code', 'name']]
df_sp500 = df_sp500.reset_index(drop=True)
df_sp500.head(5)


,code,name
0,ABT,Abbott Laboratories
1,ABBV,AbbVie
2,ACN,Accenture
3,ADBE,Adobe Inc.
4,AMD,Advanced Micro Devices Inc.


In [75]:
df_nsdq100 = pd.read_csv('NSDQ100.csv', sep=';')
df_nsdq100 = df_nsdq100[['code', 'name']]
df_nsdq100 = df_nsdq100.reset_index(drop=True)
df_nsdq100.head(5)

,code,name
0,AAPL,Apple Inc.
1,ADBE,Adobe Inc.
2,ADI,Analog Devices Inc.
3,ADP,Automatic Data Processing Inc.
4,ADSK,Autodesk Inc.


In [76]:
df_hahg = pd.read_csv('HangSeng.csv', sep=';')
df_hahg = df_hahg[['code', 'name']]
df_hahg = df_hahg.reset_index(drop=True)
df_hahg.head(5)

,code,name
0,0011.HK,Hang Seng Bank Limited
1,0388.HK,Hong Kong Exchanges and Clearing Limited
2,0939.HK,China Construction Bank Corporation
3,1299.HK,AIA Group Limited
4,1398.HK,Industrial & Commercial Bank of China Ltd.


In [77]:
data_tickers = pd.concat([df_sp500, df_nsdq100, df_hahg, df_stox])
data_tickers = data_tickers.drop_duplicates(subset=['code'])
data_tickers = data_tickers.reset_index(drop=True)
data_tickers.head()

,code,name
0,ABT,Abbott Laboratories
1,ABBV,AbbVie
2,ACN,Accenture
3,ADBE,Adobe Inc.
4,AMD,Advanced Micro Devices Inc.


In [78]:
df['tickers'] = np.nan
df

,date,content,negative,neutral,positive,entities,tickers
0,2024-04-24,Boeing has been mired in a safety crisis since...,0.981807,0.017569,0.000624,[boeing],NaN
1,2024-04-24,Families of the victims of two fatal Boeing 73...,0.653259,0.345061,0.001681,"[u, u, ., ., s, s, . justice department]",NaN
2,2024-04-24,"Merle Meyers, who left Boeing last year after ...",0.000208,0.999556,0.000236,[boeing],NaN
3,2024-04-24,Japan Airlines will continue using both Airbus...,0.000074,0.999808,0.000118,"[japan airlines, airbus, airbus, boeing]",NaN
4,2024-04-24,"Bert Subin, Aerospace Analyst at Stifel, gives...",0.000277,0.000226,0.999497,"[st, st, ##ife, ##ife, ##l, ##l, boeing]",NaN
...,...,...,...,...,...,...,...
9018,2016-02-26,"NEW YORK, NY--(Marketwired - February 26, 2016...",0.000131,0.999807,0.000062,"[& ##insky, usd, usd, ##c, ##c, boeing, boeing...",NaN
9019,2016-02-24,"NEW YORK, Feb. 24, 2016 (GLOBE NEWSWIRE) -- ...",0.000141,0.999800,0.000060,"[globe news ##wire, levi & ko ##rs ##insky, us...",NaN
9020,2016-02-24,"SAN FRANCISCO, Feb. 24, 2016 (GLOBE NEWSWIRE...",0.857394,0.141325,0.001281,"[globe news, hagen, hagen, ##s berman sob ##ol...",NaN
9021,2016-02-22,"NEW YORK, Feb. 22, 2016 (GLOBE NEWSWIRE) -- ...",0.000274,0.999333,0.000393,"[globe news ##wire, po, po, ##mer, ##mer, ##an...",NaN


In [79]:
data_tickers['name'] = data_tickers['name'].str.lower()
data_tickers.head(5)

,code,name
0,ABT,abbott laboratories
1,ABBV,abbvie
2,ACN,accenture
3,ADBE,adobe inc.
4,AMD,advanced micro devices inc.


In [114]:
data_tickers.to_csv('data_tickers.csv', index=False) 

In [80]:
data_tickers_name = data_tickers['name'].to_list()
data_tickers_name

['abbott laboratories',
 'abbvie',
 'accenture',
 'adobe inc.',
 'advanced micro devices inc.',
 'advance auto parts',
 'aes corp',
 'aflac',
 'agilent technologies, inc.',
 'air products & chemicals',
 'akamai technologies',
 'alaska air group',
 'albemarle corporation',
 'alexandria real estate equities',
 'align technology inc.',
 'allegion',
 'alliant energy',
 'allstate corp',
 'alphabet inc. (class a)',
 'alphabet inc. (class c)',
 'altria group',
 'amazon inc.',
 'amcor',
 'ameren corp',
 'american airlines group',
 'american electric power co inc.',
 'american express',
 'american international group',
 'american tower',
 'american water works',
 'ameriprise financial',
 'cencora, inc. common stock',
 'ametek',
 'amgen inc.',
 'amphenol corp',
 'analog devices inc.',
 'ansys inc.',
 'elevance health, inc. common stock',
 'aon',
 'a. o. smith',
 'apa corporation',
 'apple inc.',
 'applied materials inc.',
 'aptiv',
 'adm',
 'arista networks',
 'arthur j. gallagher & co.',
 'assu

In [116]:
len(data_tickers_name)

1537

In [81]:
ticker_boeing = data_tickers[data_tickers['code'] == 'BA']
ticker_boeing

,code,name
66,BA,boeing company


In [82]:
japan_airlines = df['entities'][3] 
japan_airlines

['japan airlines', 'airbus', 'airbus', 'boeing']

In [83]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [140]:
ticker_boeing['name']

10    boeing company
Name: name, dtype: object

In [141]:
a = process.extract(str(ticker_boeing['name']), japan_airlines, limit=2)
# Параметр limit по умолчанию имеет значение 5
print(a)

[('boeing', 90), ('japan airlines', 39)]


## Pipeline

In [84]:
data_tickers.head(5)

,code,name
0,ABT,abbott laboratories
1,ABBV,abbvie
2,ACN,accenture
3,ADBE,adobe inc.
4,AMD,advanced micro devices inc.


In [85]:
a = process.extract(str(ticker_boeing['name']), japan_airlines, limit=5)
# Параметр limit по умолчанию имеет значение 5
print(a)

[('boeing', 90), ('japan airlines', 39), ('airbus', 15), ('airbus', 15)]


In [86]:
df = df.reset_index(drop=True)

In [87]:
for index in range(len(df)):
    # Выполняем fuzzy matching для извлечения потенциальных компаний
    extraction_results = process.extract(str(df['entities'][index]), data_tickers_name, limit=5)

    # Фильтруем компании со score >= 75
    matching_companies = [company for company, score in extraction_results if score >= 87]

    if matching_companies:
        # Получаем коды компаний, соответствующие matching_companies
        matching_codes = [data_tickers[data_tickers['name'] == company]['code'].values[0] for company in matching_companies]

        codes_str = ", ".join(matching_codes)

        df['tickers'].iloc[index] = codes_str

/var/folders/yj/nhkf3nv505d5kh0dsk42f3ym0000gn/T/ipykernel_19637/4253258387.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tickers'].iloc[index] = codes_str
/var/folders/yj/nhkf3nv505d5kh0dsk42f3ym0000gn/T/ipykernel_19637/4253258387.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'BA' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df['tickers'].iloc[index] = codes_str


In [88]:
df['tickers'].value_counts()

tickers
BA                         1492
UAL                          96
LUV                          38
AIR.PA                       22
AMR                          22
                           ... 
CBH                           1
DG, GM                        1
AXP, AMT, BAC, BDX, CMA       1
SHEL, SHELL.AS                1
DOV                           1
Name: count, Length: 81, dtype: int64

## Валидация

In [89]:
df_val = pd.read_csv('BA_news_val.csv')
df_val = df_val[['date', 'title', 'content', 'tickersMentioned']]
df_val = df_val.dropna(subset=['tickersMentioned'])
df_val

,date,title,content,tickersMentioned
0,2024-01-10,UPDATE 1-Boeing hits 2023 jet delivery goal bu...,"(Recasts with Airbus comparison, adds market s...","['AIR.F', 'AIR.MC', 'AIR.PA', 'AIR.XETRA', 'AI..."
10,2024-01-10,Bitcoin’s Respectability Takes a Fresh Hit Wit...,Boeing CEO acknowledges ‘mistake’ after Alaska...,"['ALK.F', 'ALK.US', 'BA.MX', 'BA.US', 'BCO.F',..."
11,2024-01-10,MORNING BID AMERICAS-Japan shines as Wall St e...,A look at the day ahead in U.S. and global mar...,"['BA.MX', 'BA.US', 'BCO.F', 'BCO.XETRA', 'BOEI..."
12,2024-01-10,PRESS DIGEST- Wall Street Journal - Jan 10,Jan 10 (Reuters) - The following are the top s...,"['BA.MX', 'BA.US', 'BCO.F', 'BCO.XETRA', 'BOEI..."
13,2024-01-10,"Futures broadly higher, Bitcoin swings after f...",Investing.com -- U.S. stock futures were mostl...,"['BA.MX', 'BA.US', 'BCO.F', 'BCO.XETRA', 'BOEI..."
...,...,...,...,...
4233,2016-02-26,"INVESTOR ALERT: Levi & Korsinsky, LLP Notifies...","NEW YORK, NY--(Marketwired - February 26, 2016...",['BA.US']
4234,2016-02-24,"SHAREHOLDER ALERT: LEVI & KORSINSKY, LLP Notif...","NEW YORK, Feb. 24, 2016 (GLOBE NEWSWIRE) -- ...",['BA.US']
4235,2016-02-24,Hagens Berman Advises The Boeing Company (NYSE...,"SAN FRANCISCO, Feb. 24, 2016 (GLOBE NEWSWIRE...",['BA.US']
4236,2016-02-22,Pomerantz Law Firm Announces the Filing of a C...,"NEW YORK, Feb. 22, 2016 (GLOBE NEWSWIRE) -- ...",['BA.US']


In [90]:
df_val['entities'] = df_val['content'].apply(find_entity)
df_val

,date,title,content,tickersMentioned,entities
0,2024-01-10,UPDATE 1-Boeing hits 2023 jet delivery goal bu...,"(Recasts with Airbus comparison, adds market s...","['AIR.F', 'AIR.MC', 'AIR.PA', 'AIR.XETRA', 'AI...","[airbus washington, reuters, reuters, boeing, ..."
10,2024-01-10,Bitcoin’s Respectability Takes a Fresh Hit Wit...,Boeing CEO acknowledges ‘mistake’ after Alaska...,"['ALK.F', 'ALK.US', 'BA.MX', 'BA.US', 'BCO.F',...","[boeing, boeing, world bank]"
11,2024-01-10,MORNING BID AMERICAS-Japan shines as Wall St e...,A look at the day ahead in U.S. and global mar...,"['BA.MX', 'BA.US', 'BCO.F', 'BCO.XETRA', 'BOEI...","[wall st, sec, sec, wall st, bank of japan, bo..."
12,2024-01-10,PRESS DIGEST- Wall Street Journal - Jan 10,Jan 10 (Reuters) - The following are the top s...,"['BA.MX', 'BA.US', 'BCO.F', 'BCO.XETRA', 'BOEI...","[reuters, reuters, wall street journal, reuter..."
13,2024-01-10,"Futures broadly higher, Bitcoin swings after f...",Investing.com -- U.S. stock futures were mostl...,"['BA.MX', 'BA.US', 'BCO.F', 'BCO.XETRA', 'BOEI...","[securities and exchange commission, x, x, boe..."
...,...,...,...,...,...
4233,2016-02-26,"INVESTOR ALERT: Levi & Korsinsky, LLP Notifies...","NEW YORK, NY--(Marketwired - February 26, 2016...",['BA.US'],"[& ko ##rs ##insky, usd, usd, ##c, ##c, boeing..."
4234,2016-02-24,"SHAREHOLDER ALERT: LEVI & KORSINSKY, LLP Notif...","NEW YORK, Feb. 24, 2016 (GLOBE NEWSWIRE) -- ...",['BA.US'],"[globe news ##wire, levi & ko ##rs ##insky, us..."
4235,2016-02-24,Hagens Berman Advises The Boeing Company (NYSE...,"SAN FRANCISCO, Feb. 24, 2016 (GLOBE NEWSWIRE...",['BA.US'],"[globe news, hagen, hagen, ##s berman sob ##ol..."
4236,2016-02-22,Pomerantz Law Firm Announces the Filing of a C...,"NEW YORK, Feb. 22, 2016 (GLOBE NEWSWIRE) -- ...",['BA.US'],"[globe news ##wire, po, po, ##mer, ##mer, ##an..."


In [91]:
extraction_results = process.extract(str(df_val['entities'][0]), data_tickers_name, limit=5)
extraction_results

[('air products & chemicals', 57),
 ('alaska air group', 57),
 ('american airlines group', 57),
 ('boeing company', 57),
 ('cadence design systems inc.', 57)]

In [93]:
df_val['tickers'] = np.nan
df_val.head(3)

,date,title,content,tickersMentioned,entities,tickers
0,2024-01-10,UPDATE 1-Boeing hits 2023 jet delivery goal bu...,"(Recasts with Airbus comparison, adds market s...","['AIR.F', 'AIR.MC', 'AIR.PA', 'AIR.XETRA', 'AI...","[airbus washington, reuters, reuters, boeing, ...",NaN
10,2024-01-10,Bitcoin’s Respectability Takes a Fresh Hit Wit...,Boeing CEO acknowledges ‘mistake’ after Alaska...,"['ALK.F', 'ALK.US', 'BA.MX', 'BA.US', 'BCO.F',...","[boeing, boeing, world bank]",NaN
11,2024-01-10,MORNING BID AMERICAS-Japan shines as Wall St e...,A look at the day ahead in U.S. and global mar...,"['BA.MX', 'BA.US', 'BCO.F', 'BCO.XETRA', 'BOEI...","[wall st, sec, sec, wall st, bank of japan, bo...",NaN


In [98]:
df_val = df_val.reset_index(drop=True)

In [102]:
extraction_results = process.extract(str(df_val['entities'][4076]), data_tickers_name, limit=5)
extraction_results

[('adobe inc.', 57),
 ('advanced micro devices inc.', 57),
 ('agilent technologies, inc.', 57),
 ('air products & chemicals', 57),
 ('alaska air group', 57)]

In [100]:
df_val.shape

(4077, 6)

In [103]:
for index in range(len(df_val)):
    # Выполняем fuzzy matching для извлечения потенциальных компаний
    extraction_results = process.extract(str(df_val['entities'][index]), data_tickers_name, limit=5)

    # Фильтруем компании со score >= 75
    matching_companies = [company for company, score in extraction_results if score >= 87]

    if matching_companies:
        # Получаем коды компаний, соответствующие matching_companies
        matching_codes = [data_tickers[data_tickers['name'] == company]['code'].values[0] for company in matching_companies]

        codes_str = ", ".join(matching_codes)

        df_val['tickers'].iloc[index] = codes_str

## Сколько строк с распознанными тикерами

In [109]:
len(df_val['tickers']) - df_val['tickers'].isna().sum()

400

In [112]:
df_val[(df_val['tickers'].notna()) & (df_val['tickers'] != 'BA')]


,date,title,content,tickersMentioned,entities,tickers
9,2024-01-09,These Stocks Are Moving the Most Today,Boeing falls after United Airlines and Alaska ...,"['BA.MX', 'BA.US', 'BCO.F', 'BCO.XETRA', 'BOEI...","[boeing, boeing, united airlines, alaska air, ...",JNPR
19,2024-01-09,These Stocks Are Moving the Most Today: Boeing...,Boeing falls after United Airlines and Alaska ...,"['2HP.F', 'BA.MX', 'BA.US', 'BCO.F', 'BCO.XETR...","[boeing, boeing, united airlines, alaska air, ...",JNPR
33,2024-01-09,Boeing CEO Says Firm Must Acknowledge Error as...,Boeing shares slipped Tuesday after Alaska Air...,"['ALK.F', 'ALK.US', 'BA.MX', 'BA.US', 'BCO.F',...","[boeing, boeing, alaska air group, united airl...","ALK, UAL"
36,2024-01-09,These Stocks Moved the Most Today: Juniper Net...,Juniper Networks stock surges following a repo...,"['2HP.F', 'BA.MX', 'BA.US', 'BCO.F', 'BCO.XETR...","[juniper networks, he, he, ##wl, ##wl, ##ett p...",JNPR
117,2024-01-06,FAA Orders Temporary Grounding of Many Boeing ...,Alaska Air Flight 1282 was able to return to t...,"['ALK.F', 'ALK.US', 'BA.MX', 'BA.US', 'BCO.F',...",[alaska air],ALK
...,...,...,...,...,...,...
3839,2021-04-07,How Much Does the US Trade With the EU?,Here's a look at what at top items traded betw...,"['BA.US', 'BOEI34.SA', 'EADSF.US', 'EADSY.US']",[eu],"WY, BUD, BIM.PA, DBK.DE, DPW.DE"
3857,2021-03-31,Dow Jones Rallies As Biden Unveils $2T Infrast...,The Dow Jones rallied after President Joe Bide...,"['BA.US', 'BOEI34.SA', 'DJI.INDX']",[apple],"AAPL, PPL"
3858,2021-03-31,"Dow Jones Up, Nasdaq Leads, As Biden Unveils $...",The Dow Jones rallied after President Joe Bide...,"['BA.US', 'BOEI34.SA', 'DJI.INDX']",[apple],"AAPL, PPL"
3869,2021-03-29,"Boeing Stock Is Jumping, With 3 Pieces of Good...","Southwest Airlines ordered more737 MAX jets, b...","['BA.US', 'BOEI34.SA']",[southwest airlines],LUV
